In [1]:
import pandas as pd

everyday_df = pd.read_csv('Cannabis_Strains_Features.csv')

In [2]:
everyday_df.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [3]:
!python -m spacy download en_core_web_lg

/bin/bash: python: command not found


In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import spacy
nlp = spacy.load("en_core_web_lg")

In [5]:
def tokenize(doc):
  lemmas = []
    
  doc = nlp(doc)
    
  for token in doc: 
    if ((token.is_stop == False) and (token.is_punct == False) and (token.pos_ != 'PRON')):
      lemmas.append(token.lemma_)
  
  return lemmas

everyday_df['Description_2'] = everyday_df['Description'].apply(tokenize)

In [6]:
everyday_df['bow_col'] = "string"
for i in range(0, len(everyday_df['Effects'])):
  everyday_df['bow_col'][i] = everyday_df['Effects'][i].split(",") + everyday_df['Flavor'][i].split(",") + everyday_df['Description_2'][i]

<ipython-input-6-21fa107fb388>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  everyday_df['bow_col'][i] = everyday_df['Effects'][i].split(",") + everyday_df['Flavor'][i].split(",") + everyday_df['Description_2'][i]


In [7]:
everyday_df.head()

,Strain,Type,Rating,Effects,Flavor,Description,Description_2,bow_col
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"[$, 100, OG, 50/50, hybrid, strain, pack, stro...","[Creative, Energetic, Tingly, Euphoric, Relaxe..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"[98, Aloha, White, Widow, especially, potent, ...","[Relaxed, Aroused, Creative, Happy, Energetic,..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"[1024, sativa, dominant, hybrid, breed, Spain,...","[Uplifted, Happy, Relaxed, Energetic, Creative..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"[13, Dawgs, hybrid, G13, Chemdawg, genetic, br...","[Tingly, Creative, Hungry, Relaxed, Uplifted, ..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","[know, Kosher, Tangie, 24k, gold, 60, indica, ...","[Happy, Relaxed, Euphoric, Uplifted, Talkative..."


In [8]:
everyday_df['bow_col'][0]

['Creative',
 'Energetic',
 'Tingly',
 'Euphoric',
 'Relaxed',
 'Earthy',
 'Sweet',
 'Citrus',
 '$',
 '100',
 'OG',
 '50/50',
 'hybrid',
 'strain',
 'pack',
 'strong',
 'punch',
 'supposedly',
 'refer',
 'strength',
 'high',
 'price',
 'start',
 'show',
 'Hollywood',
 'plant',
 '$',
 '100',
 'OG',
 'tend',
 'produce',
 'large',
 'dark',
 'green',
 'bud',
 'stem',
 'user',
 'report',
 'strong',
 'body',
 'effect',
 'indica',
 'pain',
 'relief',
 'alert',
 'cerebral',
 'feeling',
 'thank',
 'sativa']

In [9]:
from collections import Counter

word_counts = Counter()
everyday_df['bow_col'].apply(lambda x: word_counts.update(x))
word_counts.most_common(10)

[('strain', 3020),
 ('  ', 2138),
 ('Happy', 1902),
 ('Relaxed', 1758),
 ('Euphoric', 1669),
 ('effect', 1596),
 ('Uplifted', 1535),
 ('indica', 1394),
 ('hybrid', 1311),
 ('Sweet', 1166)]

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words=nlp.Defaults.stop_words, max_features=1000)
vect.fit(everyday_df['bow_col'].astype('str'))
dtm = vect.transform(everyday_df['bow_col'].astype('str'))
dtm = pd.DataFrame(dtm.todense(),columns=vect.get_feature_names())
dtm.head()

/home/shuulaces/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,10,11,12,13,14,15,18,1st,20,2012,...,woody,work,world,worth,wrap,wreck,xa0,year,yield,zesty
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.121432,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.117434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.621151,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.115823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.151705,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', n_neighbors=10)

In [12]:
random_pot = ["I want something to take the edge off of my day, really mellow me out and help get me in the zone"]

In [13]:
new = vect.transform(random_pot)

In [14]:
nn.kneighbors(new.todense())

(array([[1.        , 1.        , 1.        , 1.2184825 , 1.22524616,
         1.25964668, 1.27678542, 1.28071757, 1.28319214, 1.2923116 ]]),
 array([[1653, 1651, 1652, 2318, 1785,  672,  166, 2251, 2068, 1124]]))

In [15]:
everyday_df['bow_col'][2068]

['Relaxed',
 'Focused',
 'Uplifted',
 'Talkative',
 'Happy',
 'Pine',
 'Earthy',
 'Spicy/Herbal',
 'Suzy',
 'Q',
 'high',
 'CBD',
 'low',
 'thc',
 'strain',
 'piney',
 'taste',
 'help',
 'treat',
 'symptom',
 'little',
 'high',
 'great',
 'strain',
 'add',
 'burgeon',
 'world',
 'CBD',
 'strain',
 'strain',
 'test',
 'upwards',
 '59:1',
 'CBD',
 'thc',
 'hybrid',
 'great',
 'daytime',
 'use',
 'want',
 'relieve',
 'chronic',
 'pain',
 'nausea',
 'arthritis',
 'muscle',
 'spasm',
 'anxiety',
 'psychoactive',
 'effect',
 '\xa0 \xa0\xa0']

In [16]:
everyday_df['Rating'].value_counts()

4.5    323
4.3    299
4.4    287
4.6    245
4.2    227
5.0    219
4.7    166
4.8    162
4.0    109
4.1    101
4.9     61
0.0     35
3.9     29
3.8     23
3.7     16
3.0     13
3.6     11
3.5     10
3.4      5
3.2      3
3.3      3
3.1      1
2.5      1
1.0      1
2.8      1
Name: Rating, dtype: int64

In [29]:
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dropout

model = Sequential()

n_nodes = 32
model.add(Dense(n_nodes, activation="relu", input_dim=1000))

n_nodes_output = 25
model.add(Dense(n_nodes_output, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

results = model.fit(dtm[:1000], 
                    everyday_df['Rating'][:1000], 
                    epochs=45, 
                    validation_data=(dtm[1000:2000], everyday_df['Rating'][1000:2000]))

#model_class = KerasClassifier(build_fn=model, verbose=1)

#param_grid = {'batch_size': [32,64,512],
#              'epochs': [3],
#              'units':[32]}

#grid = GridSearchCV(estimator=model_class, 
#                    param_grid=param_grid, 
#                    n_jobs=-1, 
#                    verbose=1)

#grid_result = grid.fit(dtm[:1000], everyday_df['Rating'][:1000])

#print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']

#for mean, stdev, param in zip(means, stds, params):
#    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/45
32/32 [==============================] - 0s 3ms/step - loss: 3.0520 - accuracy: 0.0360 - val_loss: 2.7945 - val_accuracy: 0.0500
Epoch 2/45
32/32 [==============================] - 0s 1ms/step - loss: 2.4590 - accuracy: 0.0410 - val_loss: 2.0063 - val_accuracy: 0.0500
Epoch 3/45
32/32 [==============================] - 0s 1ms/step - loss: 1.6032 - accuracy: 0.0410 - val_loss: 1.1495 - val_accuracy: 0.0500
Epoch 4/45
32/32 [==============================] - 0s 1ms/step - loss: 0.9653 - accuracy: 0.0410 - val_loss: 0.7500 - val_accuracy: 0.0500
Epoch 5/45
32/32 [==============================] - 0s 1ms/step - loss: 0.7498 - accuracy: 0.0410 - val_loss: 0.6453 - val_accuracy: 0.0500
Epoch 6/45
32/32 [==============================] - 0s 1ms/step - loss: 0.6842 - accuracy: 0.0410 - val_loss: 0.6091 - val_accuracy: 0.0500
Epoch 7/45
32/32 [==============================] - 0s 1ms/step - loss: 0.6494 - accuracy: 0.0410 - val_loss: 0.5899 - val_accuracy: 0.0500
Epoch 8/45
32/32 [==

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                32032     
_________________________________________________________________
dense_5 (Dense)              (None, 25)                825       
Total params: 32,857
Trainable params: 32,857
Non-trainable params: 0
_________________________________________________________________
